In [1]:
# !pip install -qqq -U git+https://github.com/huggingface/transformers.git
# !pip install -qqq -U git+https://github.com/huggingface/peft.git
# !pip install -qqq -U git+https://github.com/huggingface/accelerate.git
# !pip install -qqq bitsandbytes
# !pip install -qqq datasets
# !pip install evaluate
# !pip install rouge_score
# !pip install torch

In [1]:
from datasets import load_dataset
dataset= load_dataset('csv', data_files = "/root/NA0219/Dataset_Formatting/Combined_Dastset_14498.csv",encoding='UTF-8')
dataset = dataset["train"].train_test_split(test_size=0.2)

Generating train split: 0 examples [00:00, ? examples/s]

In [2]:
train_data= dataset["train"]
test_data= dataset["test"]

In [3]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
import numpy as np
import evaluate
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [4]:
# base_model_id = "HuggingFaceH4/zephyr-7b-alpha"
base_model_id = "HuggingFaceH4/zephyr-7b-beta"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config, device_map="auto")

/usr/local/lib/python3.8/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

...

model-00006-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [5]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model_id,
    model_max_length=784,
    padding_side="left",
    add_eos_token=True)

tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

In [6]:
def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=784,
        padding="max_length",
    )
    result["labels"] = result["input_ids"].copy()
    return result

In [7]:
def generate_and_tokenize_prompt(data_point):
    full_prompt = f"""Extract specific data from the provided OCR text of a marriage record. Identify and return the following details in a structured JSON format: Event place, event date (day, month, year), marriage date (day, month, year), personal details of both parties including their names (given and surname), age, birth date (day, month, year), and parents' names (given and surname). Only show the extracted information. Don't show the OCR Text again. Also don't show the pattern which you are asked to follow. Concentrate on the data asked to extract. The output must strictly adhere to the given details, starting with an opening curly brace '{{' and ending with a closing curly brace '}}'. Ensure that each key-value pair contains only accurate and relevant information from the OCR text. Avoid including any extraneous or irrelevant data that does not correspond directly to the specified keys. The structure of the output is as follows:

Event_Day_orig: The original day of the event.
Event_Month_orig: The original month of the event.
Event_Year_orig: The original year of the event.
Given_Name_orig: The given name of the person.
Surname_orig: The surname of the person.
Sex_orig: The gender of the person.
Age_orig: The age of the person.
Birth_Year_orig: The original year of birth of the person.
Death_Year_orig: The original year of death of the person.
Father_Given_Name_orig: The given name of the person's father.
Father_Surname_orig: The surname of the person's father.
Mother_Given_Name_orig: The given name of the person's mother.
Mother_Surname_orig: The surname of the person's mother.
Spouse_Given_Name_orig: The given name of the spouse.
Spouse_Surname_orig: The surname of the spouse.
Spouse_Age_orig: The age of the spouse.
Spouse_Birth_Year_orig: The original year of birth of the spouse.
Spouse_Father_Given_Name_orig: The given name of the spouse's father.
Spouse_Father_Surname_orig: The surname of the spouse's father.
Spouse_Mother_Given_Name_orig: The given name of the spouse's mother.
Spouse_Mother_Surname_orig: The surname of the spouse's mother.


### OCR Text:
{data_point["Input"]}

### JSON Output:
{data_point["Output"]}
"""

    return tokenize(full_prompt)

In [8]:
tokenized_train_dataset = train_data.map(generate_and_tokenize_prompt)
tokenized_val_dataset = test_data.map(generate_and_tokenize_prompt)

Map:   0%|          | 0/11598 [00:00<?, ? examples/s]

Map:   0%|          | 0/2900 [00:00<?, ? examples/s]

In [10]:
# eval_prompt = """Extract specific data from the provided OCR text of a marriage record. Identify and return the following details in a structured JSON format: Application number, date of marriage ( month,day, year), place of marriage, birth details (month,day, year, place) of both spouses, their county, gender, given names, residence, state, and surnames. Only show the extracted information. Don't show the OCR Text again. Also don't show the pattern which you are asked to follow. Concentrate on the data asked to extract . The output must strictly adhere to the given details, starting with an opening curly brace '{' and ending with a closing curly brace '}'. Ensure that each key-value pair contains only accurate and relevant information from the OCR text. Avoid including any extraneous or irrelevant data that does not correspond directly to the specified keys. The structure of the output is as follows:

# Application#: The marriage Application number.
# Spouse1_given: The first name of the first spouse.
# Spouse1_surname: The last name of the first spouse.
# Spouse1_birth_day: The birth day of the first spouse.
# Spouse1_birth_month: The birth month of the first spouse.
# Spouse1_birth_year: The birth year of the first spouse.
# Spouse1_residence: The residence of the first spouse.
# Spouse1_county: The county of the first spouse.
# Spouse1_state: The state of the first spouse.
# Spouse1_birthplace: The birthplace of the first spouse.
# Spouse1_gender: The gender of the first spouse.
# Spouse2_given: The first name of the second spouse.
# Spouse2_surname: The last name of the second spouse.
# Spouse2_maiden: The maiden name of the second spouse, if applicable.
# Spouse2_birth_day: The birth day of the second spouse.
# Spouse2_birth_month: The birth month of the second spouse.
# Spouse2_birth_year: The birth year of the second spouse.
# Spouse2_residence: The residence of the second spouse.
# Spouse2_county: The county of the second spouse.
# Spouse2_state: The state of the second spouse.
# Spouse2_birthplace: The birthplace of the second spouse.
# Spouse2_gender: The gender of the second spouse.
# Marriage_day: The day of the marriage.
# Marriage_month: The month of the marriage.
# Marriage_year: The year of the marriage.
# Marriage_place: The location of the marriage.

# ### OCR Text:
# UNCERTIFI COPY 1. GROOM'S NAME (First, Middle, Last) 5a. BRIDE'S NAME CLERK, HIL RIVERVIEW CLERK HILL CHARLES WILLIAM GOSS RESIDENCE-CITY TOWN OR LOCATION CIRCUIT 7a. RESIDENCE - CITY, TOWN, OR LOCATION RIVERVIEW. Middle, Last) COLLEEN MARIE GALLIVAN Department of Health Vital Statistics STATE OF FLORIDA MARRIAGE RECORD TYPE IN UPPER CASE USE BLACK INK This license not valid unless seal of Clerk. Circuit or County Court. appears thereon. OPY 2004 32817 SEAL COURT LSBOROUG COURT (APPLICATION NUMBER) UNCERTIFIED PY â¢ M. APPLICATION TO MARRY ERT Co 3b. COUNTY HILLSBOROUGH 7b. COUNTY TITLE OF OFFICIAL BARBARA B KING DEPUTY CLERK 3. SIGNATURE OF BRIDE (Sign full name using black ink) 9. SIGNATUREOF GROOM (San - Charbstfoss CERTIFI all 15. TITLE OF OFFICIAL BARBARA B KING DEPUTY CLERK UNDERTIFIED OPY UNCERTIFI OPY HILLSBOROUGH 08. SIATURE OF COURT CEK OR MOGE 23a SIGNATURE OF PERSON PERFORMING CEREMONY (Use black ink) ME. Anderson Book685/Page 179 BOOK 685 PAGE 179 23b. NAME AND TTLE OF PEASFORPHPALLS EMANDERSON MY COMMISSION #DD 289869 EXPIRES: May 5, 2008 Bonded Thru Budget Hotary Services 3c. STATE HILLSBOROUGH FLORIDA WE THE APPLICANTS NAMED IN THIS CERTIFICATE, EACH FOR HIMSELF OR HERSELF, STATE THAT THE INFORMATION PROVIDED ON THIS RECORD IS CORRECT TO THE BEST OF OUR KNOWLEDGE AND BELIEF, THAT NO LEGAL OBJECTION TO THE MARRIAGE NOR THE ISSUANCE OF A LICENSE TO AUTHORIZE THE SAME IS KNOWN TO US AND HEREBY APPLY FOR LICENSE TO MARRY. name using black ink) 10. SUBSCRIBED AND SWORN TO BEFORE ME ON (DATE) 11/22/2004 12. SIGNATURE OF OFFICIAL (Use black) FLORIDA 5b. MAIDEN SURNAME (If 7c. STATE LICENSE TO MARRY AUTHORIZATION AND LICENSE IS HEREBY GIVEN TO ANY PERSON DULY AUTHORIZED BY THE LAWS OF THE STATE OF FLORIDA TO PERFORM A MARRIAGE CEREMONY WITHIN THE STATE OF FLORIDA AND TO SOLEMNIZE THE MARRIAGE OF THE ABOVE NAMED PERSONS. THIS LICENSE MUST BE USED ON OR AFTER THE EFFECTIVE DATE AND ON OR BEFORE THE EXPIRATION DATE IN THE STATE OF FLORIDA IN ORDER TO BE RECORDED AND VALID. COUNTY ISSUING LICENSE 18. DATE LICENSE ISSUED 18a. DATE LICENSE EFFECTIVE 19. EXPIRATION DATE 11/22/2004 co A CERTIFICATE OF MARRIAGE 22. CITY, TOWN, OR LOCATION OF MARRIAGE I HEREBY CERTIFY THAT THE ABOVE NAMED GROOM AND BRIDE WERE JOINED BY ME IN MARRIAGE IN ACCORDANCE WITH THE LAWS OF THE STATE OF FLORIDA 21. DAT OF MARRIAGE (Month, Day, Year) JACKSON ville UMBERTIFIED COPY 20b. TITLE DATE OF BIRTH (Month, Day, Year) 1/25/1954 4. BIRTHPLACE (State or Foreign Country) NEW YORK 6. DATE OF DEPUTY CLERK the 14. SUBSCRIBED AND SWORN TO BEFOREXIE ON DATE) 11/22/2004 16. SIGNATRE OF OFFICIAL (Use black ink) Lore 9/7/1963 8. BIRTHPLACE (State or Foreign Country) NEW YORK H (Month, Day, 11/25/2004 RTIFIED ATE OF FLOW INFORMATION BELOW FOR USE BY VITAL STATISTICS ONLY - NOT TO BE RECORDED COPY UNCERTIFIED COPY CERTIFIAL P. Anderson 23c. ADDRESS (of person 335 River Bluff Pd. N. Jacksonville, FL 24. SIGNATURE OF WITNESS TO CERE 11 (se black ink) 1/24/2005 CERTIFIED 25. SIGNATURE OF WITNESS TO CEREMONY (Use black ink) 20c. BY D.C BBK COPY UNCERTIFIED COPY.
# """

In [9]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [10]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [11]:
from peft import LoraConfig, get_peft_model
config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 21260288 || all params: 3773331456 || trainable%: 0.5634354746703705


In [12]:
import transformers
from datetime import datetime

project = "finetune-14498_NA0219"
base_model_name = "zephyr_7B_beta"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name

tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=5,
        per_device_train_batch_size=1,
        gradient_checkpointing=True,
        gradient_accumulation_steps=1,
        max_steps=14000,
        learning_rate=1.5e-5,
        logging_steps=2800,
        bf16=False,
        optim="paged_adamw_32bit",
        logging_dir="./logs",
        save_strategy="steps",
        save_steps=2800,
        evaluation_strategy="steps",
        eval_steps=2800,
        do_eval=True,
        run_name=f"{run_name}-{datetime.now().strftime('%Y-%m-%d-%H-%M')}"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/training_args.py:1463: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.8/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
2800,1.770900,1.613042
5600,1.544900,1.531480
8400,1.514300,1.489800
11200,1.471800,1.463081
14000,1.417600,1.451565


/usr/local/lib/python3.8/dist-packages/peft/utils/save_and_load.py:180: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/usr/local/lib/python3.8/dist-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/peft/utils/save_and_load.py:180: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found i

TrainOutput(global_step=14000, training_loss=1.5439128941127231, metrics={'train_runtime': 46575.8583, 'train_samples_per_second': 0.301, 'train_steps_per_second': 0.301, 'total_flos': 4.69679748808704e+17, 'train_loss': 1.5439128941127231, 'epoch': 1.2071046732195205})